In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score
from sklearn.impute import SimpleImputer

In [2]:
# Load the dataset
data = pd.read_csv('SampleFile2.csv')

# Convert the date column to datetime format
data['Date'] = pd.to_datetime(data['Date'])

# Filter the data to include only the previous two years before 04-29-2024
cutoff_date = pd.Timestamp('2024-04-29')
start_date = cutoff_date - pd.DateOffset(years=2)
data = data[(data['Date'] >= start_date) & (data['Date'] < cutoff_date)]

C:\Users\AbdulRafay\AppData\Local\Temp\ipykernel_3440\3967702544.py:2: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('SampleFile2.csv')


In [3]:
# Drop columns that are not useful for the model
columns_to_drop = ['N+C', 'Region', 'Channel', 'LI', 'Category', 'Class', 'Sub-Class', 'Season', 
                   'Color', 'Size', 'Size1', 'Color1', 'Discount Tag', 'Fiscal Year', 'Invoice Date']


In [4]:
# Remove rows with non-numeric 'Sales Amount'
data = data[pd.to_numeric(data['Sales Amount'], errors='coerce').notnull()]

# Drop the specified columns
data = data.drop(columns=columns_to_drop)

In [5]:
# Extract time-based features
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
data['Day'] = data['Date'].dt.day
data['DayOfWeek'] = data['Date'].dt.dayofweek
data['WeekOfYear'] = data['Date'].dt.isocalendar().week


In [8]:
# Remove rare 'Product Name' categories with only one instance
product_name_counts = data['Product Name'].value_counts()
rare_product_names = product_name_counts[product_name_counts == 1].index
data = data[~data['Product Name'].isin(rare_product_names)]

In [21]:
# Save original 'Product Name' and 'Shop' for analysis later
original_product_name = data['Product Name']
original_shop = data['Shop']


In [9]:
# Define features and target
target = 'Sales Amount'
features = [col for col in data.columns if col != target]

In [10]:
# Split the data into train and test sets stratified by 'Product Name'
stratified_split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, test_idx = next(stratified_split.split(data, data['Product Name']))

train_data = data.iloc[train_idx]
test_data = data.iloc[test_idx]

In [11]:
# One-hot encode important categorical columns
important_categorical_columns = ['Product Name', 'Shop', 'Gender']
train_data = pd.get_dummies(train_data, columns=important_categorical_columns)
test_data = pd.get_dummies(test_data, columns=important_categorical_columns)

In [12]:
# Ensure both train and test sets have the same columns after encoding
train_data, test_data = train_data.align(test_data, join='outer', axis=1, fill_value=0)


In [13]:
# Define features and target for training and testing
X_train = train_data.drop(columns=['Date', target])
y_train = train_data[target]
X_test = test_data.drop(columns=['Date', target])
y_test = test_data[target]

In [14]:
# Handle missing values
imputer = SimpleImputer(strategy='mean')
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

In [15]:
# Ensure there is enough data to train the model
if len(X_train) == 0 or len(X_test) == 0:
    raise ValueError("Insufficient data to train the model. Adjust the train/test split or check your dataset.")


In [16]:
# Train the Random Forest model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)


RandomForestRegressor(random_state=42)

In [17]:
# Make predictions
y_pred = model.predict(X_test)

In [18]:
# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Absolute Error: {mae}')
print(f'Mean Absolute Percentage Error: {mape}')
print(f'R2 Score: {r2}')

Mean Absolute Error: 4.876858458659295
Mean Absolute Percentage Error: 0.0013656320515773677
R2 Score: 0.99858761495329


In [19]:
# Add predictions to the test set for analysis
test_data['Predicted Sales Amount'] = y_pred

In [22]:
# Restore original 'Product Name' and 'Shop' columns for analysis
test_data['Product Name'] = original_product_name.iloc[test_idx].values
test_data['Shop'] = original_shop.iloc[test_idx].values

In [23]:
# Group by Product Name and Shop to get actual and predicted sales
actual_sales_product = test_data.groupby('Product Name')['Sales Amount'].sum().reset_index()
predicted_sales_product = test_data.groupby('Product Name')['Predicted Sales Amount'].sum().reset_index()

actual_sales_shop = test_data.groupby('Shop')['Sales Amount'].sum().reset_index()
predicted_sales_shop = test_data.groupby('Shop')['Predicted Sales Amount'].sum().reset_index()


In [24]:
# Calculate errors
product_errors = actual_sales_product.merge(predicted_sales_product, on='Product Name')
product_errors['Error'] = np.abs(product_errors['Sales Amount'] - product_errors['Predicted Sales Amount'])

shop_errors = actual_sales_shop.merge(predicted_sales_shop, on='Shop')
shop_errors['Error'] = np.abs(shop_errors['Sales Amount'] - shop_errors['Predicted Sales Amount'])


In [25]:
# Identify combinations with lowest and highest errors
lowest_error_product = product_errors.sort_values(by='Error').head(5)
highest_error_product = product_errors.sort_values(by='Error').tail(5)

lowest_error_shop = shop_errors.sort_values(by='Error').head(5)
highest_error_shop = shop_errors.sort_values(by='Error').tail(5)

print("Lowest Error Products:\n", lowest_error_product)
print("Highest Error Products:\n", highest_error_product)
print("Lowest Error Shops:\n", lowest_error_shop)
print("Highest Error Shops:\n", highest_error_shop)

Lowest Error Products:
     Product Name  Sales Amount  Predicted Sales Amount  Error
262    AHB-S24-7       1500.00                 1500.00    0.0
47     ABR-S24-7       8389.80                 8389.80    0.0
242   AER-W23-50       3355.92                 3355.92    0.0
52    ABR-W23-10       2516.94                 2516.94    0.0
264    AHB-S24-9       1000.00                 1000.00    0.0
Highest Error Products:
        Product Name  Sales Amount  Predicted Sales Amount        Error
705      MB-F24-509     112956.59           108250.854100  4705.735900
1131      MB-W22-91     503569.59           498710.063118  4859.526882
610     MB-EF24-114     546730.38           552121.324500  5390.944500
642     MB-EF24-191     145374.98           151088.800100  5713.820100
936   MB-US23-109-A     364972.09           359048.664865  5923.425135
Lowest Error Shops:
                        Shop  Sales Amount  Predicted Sales Amount  \
18       Kartarpur Corridor      32151.30            32151.3000

In [26]:
# Compute error metrics for each Shop and Product Name combination in the test set
test_data['Error'] = np.abs(test_data['Sales Amount'] - test_data['Predicted Sales Amount'])
error_metrics = test_data.groupby(['Shop', 'Product Name']).agg(
    Samples=('Sales Amount', 'count'),
    MAE=('Error', 'mean'),
    MAPE=('Error', lambda x: np.mean(np.abs(x / test_data['Sales Amount']))),
    R2=('Error', lambda x: r2_score(test_data['Sales Amount'], test_data['Predicted Sales Amount']))
).reset_index()

# Save error metrics to CSV
error_metrics.to_csv('error_metrics.csv', index=False)